In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("datafiniti/consumer-reviews-of-amazon-products")

print("Path to dataset files:", path)

100%|██████████| 16.3M/16.3M [00:00<00:00, 71.7MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/datafiniti/consumer-reviews-of-amazon-products/versions/5


In [ ]:
import pandas as pd
import os
import seaborn as sns

In [ ]:
os.listdir("/root/.cache/kagglehub/datasets/datafiniti/consumer-reviews-of-amazon-products/versions/5")

['1429_1.csv',
 'Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products.csv',
 'Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv']

In [ ]:
data = pd.concat(
    [
      pd.read_csv(os.path.join(path, "Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv")),
      pd.read_csv(os.path.join(path, "Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products.csv")),
      pd.read_csv(os.path.join(path, "1429_1.csv"))
    ]
)

<ipython-input-4-39b007442c8f>:5: DtypeWarning: Columns (1,10) have mixed types. Specify dtype option on import or set low_memory=False.
  pd.read_csv(os.path.join(path, "1429_1.csv"))


In [ ]:
data = data[['id', 'name', 'reviews.doRecommend','reviews.rating', 'reviews.text']]

In [ ]:
len(data), len(data.drop_duplicates())

(67992, 64303)

In [ ]:
data = data.drop_duplicates()

In [ ]:
data = data[(data['name'].isna()==False) & (data['reviews.rating'].isna()==False) & (data['reviews.text'].isna()==False)]

In [ ]:
data.reset_index(inplace=True, drop=True)

In [ ]:
def split_reviews_by_rating(df):
    result = {}
    for product_id, group in df.groupby('id'):
        low_rating_reviews = group[group['reviews.rating'] < 3]['reviews.text'].tolist()
        high_rating_reviews = group[group['reviews.rating'] > 3]['reviews.text'].tolist()
        result[product_id] = {
            'name': group['name'].iloc[0],
            'low_rating_reviews': low_rating_reviews,
            'high_rating_reviews': high_rating_reviews
        }
    return result

reviews_by_product = split_reviews_by_rating(data)

# Dpo data gen

In [ ]:
exclude = [k for k, p in reviews_by_product.items() if len(p["low_rating_reviews"])<28]

In [ ]:
sum([len(p["low_rating_reviews"]) for k, p in reviews_by_product.items()]) / len([len(p["low_rating_reviews"]) for k, p in reviews_by_product.items()])

28.19753086419753

In [ ]:
all_pairs_by_product = {}
for product in reviews_by_product.keys():
  if product not in exclude:
    all_pairs_by_product[reviews_by_product[product]['name']] = []
    j = 0
    for pos_review in reviews_by_product[product]['high_rating_reviews']:
        all_pairs_by_product[reviews_by_product[product]['name']].append([reviews_by_product[product]['low_rating_reviews'][j], pos_review])
        j = (j + 1) % len(reviews_by_product[product]['low_rating_reviews'])

In [ ]:
{p['name']: len(p["low_rating_reviews"]) for p in reviews_by_product.values()}["All-New Kindle Oasis E-reader - 7 High-Resolution Display (300 ppi), Waterproof, Built-In Audible, 8 GB, Wi-Fi - Includes Special Offers"]

1

In [ ]:
dpo_data = []
for k, v in all_pairs_by_product.items():
  df = pd.DataFrame({
      "name": k,
      "items":v
  })
  dpo_data.append(df)

In [ ]:
dpo_data = pd.concat(dpo_data)

In [ ]:
dpo_data['prompt'] = dpo_data['name'].apply(lambda x: [{
    "content": "Write a negative review on product with name " + x,
    "role": "user"
}])

In [ ]:
dpo_data['completion'] = dpo_data['items'].apply( lambda x:
    {
        "chosen": [{
    "content": x[0],
    "role": "assistant"
}],
         "rejected": [{
    "content": x[1],
    "role": "assistant"
}]
    }
)

In [ ]:
len(dpo_data)

45171

In [ ]:
dpo_data[['prompt', 'completion']].to_csv("dpo_dataset.csv", index=False)

In [ ]:
dpo_data

,name,items,prompt,completion
0,Amazon Kindle Paperwhite - eBook reader - 4 GB...,"[Reading a book, it's great. Working with PDF ...",[{'content': 'Write a negative review on produ...,"{'chosen': [{'content': 'Reading a book, it's ..."
1,Amazon Kindle Paperwhite - eBook reader - 4 GB...,[I am really disappointed in this Kindle....I ...,[{'content': 'Write a negative review on produ...,{'chosen': [{'content': 'I am really disappoin...
2,Amazon Kindle Paperwhite - eBook reader - 4 GB...,[Paper white does not allow you do use books f...,[{'content': 'Write a negative review on produ...,{'chosen': [{'content': 'Paper white does not ...
3,Amazon Kindle Paperwhite - eBook reader - 4 GB...,[Tried to use it for and it's does not work as...,[{'content': 'Write a negative review on produ...,{'chosen': [{'content': 'Tried to use it for a...
4,Amazon Kindle Paperwhite - eBook reader - 4 GB...,[Our daughter loves the size and portability o...,[{'content': 'Write a negative review on produ...,{'chosen': [{'content': 'Our daughter loves th...
...,...,...,...,...
339,Amazon 5W USB Official OEM Charger and Power A...,[Really expected a chord with it for the price...,[{'content': 'Write a negative review on produ...,{'chosen': [{'content': 'Really expected a cho...
340,Amazon 5W USB Official OEM Charger and Power A...,[Not compatible with South African plug standa...,[{'content': 'Write a negative review on produ...,{'chosen': [{'content': 'Not compatible with S...
341,Amazon 5W USB Official OEM Charger and Power A...,[It wasn't what I thought it was the price was...,[{'content': 'Write a negative review on produ...,{'chosen': [{'content': 'It wasn't what I thou...
342,Amazon 5W USB Official OEM Charger and Power A...,"[I had to buy it, what else is there to say., ...",[{'content': 'Write a negative review on produ...,"{'chosen': [{'content': 'I had to buy it, what..."


# Kto data gen

In [ ]:
kto_data = data[(data['reviews.rating'] != 3) & (~data['id'].isin(exclude))]

In [ ]:
kto_data['target'] = kto_data['reviews.rating'].apply(lambda x: True if x < 3 else False)

<ipython-input-122-eb99b19baa13>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kto_data['target'] = kto_data['reviews.rating'].apply(lambda x: True if x < 3 else False)


In [ ]:
kto_data = kto_data[['name', 'reviews.text', 'target']]

In [ ]:
kto_data = kto_data.rename(
    columns={
        "name": "product_name",
        "reviews.text": "text",
    }
)

In [ ]:
kto_data['product_name'] = kto_data['product_name'].apply(lambda x: [{
    "content": "Write a negative review on product with name " + x,
    "role": "user"
}])

In [ ]:
kto_data['text'] = kto_data['text'].apply(lambda x: [{
    "content": x,
    "role": "assistant"
}])

In [ ]:
kto_data = kto_data.rename(
    columns={
        "product_name": "prompt",
        "text": "completion",
    }
)

In [ ]:
kto_data.to_csv('kto_dataset.csv', index=False)

In [ ]:
kto_data

,prompt,completion,target
1,[{'content': 'Write a negative review on produ...,[{'content': 'Bulk is always the less expensiv...,False
2,[{'content': 'Write a negative review on produ...,[{'content': 'Well they are not Duracell but f...,False
3,[{'content': 'Write a negative review on produ...,[{'content': 'Seem to work as well as name bra...,False
4,[{'content': 'Write a negative review on produ...,[{'content': 'These batteries are very long la...,False
5,[{'content': 'Write a negative review on produ...,[{'content': 'Bought a lot of batteries for Ch...,False
...,...,...,...
57503,[{'content': 'Write a negative review on produ...,[{'content': 'Just love it. Easy to set up and...,False
57505,[{'content': 'Write a negative review on produ...,[{'content': 'This is my new favorite device. ...,False
57506,[{'content': 'Write a negative review on produ...,[{'content': 'I got this to basically experime...,False
57507,[{'content': 'Write a negative review on produ...,[{'content': 'Good product that does the basic...,False
